# NLP demo software by HyperbeeAI

Copyrights © 2023 Hyperbee.AI Inc. All rights reserved. hello@hyperbee.ai 

### Deployment

This notebook acts as the serial terminal that we use in the ai85 translation demo.

- load parameter set
- run a test on the PC to determine what to expect from the chip
- run test on the chip via serial terminal on PC

### Initialization

In [1]:
import torch, random
import numpy as np
import torch.nn as nn
from torchtext.legacy.datasets import TranslationDataset
from torchtext.legacy.data     import Field, BucketIterator
from utils      import tokenize_es, tokenize_en, tokenizer_es, tokenizer_en, TRG_PAD_IDX, \
                       translate_sentence, calculate_bleu, license_statement
from models     import encoder, decoder, seq2seq
from dataloader import NewsDataset

import serial

imported utils.py
NLP demo software by HyperbeeAI. Copyrights © 2023 Hyperbee.AI Inc. All rights reserved. hello@hyperbee.ai

imported layers.py
NLP demo software by HyperbeeAI. Copyrights © 2023 Hyperbee.AI Inc. All rights reserved. hello@hyperbee.ai

imported functions.py
NLP demo software by HyperbeeAI. Copyrights © 2023 Hyperbee.AI Inc. All rights reserved. hello@hyperbee.ai

imported models.py
NLP demo software by HyperbeeAI. Copyrights © 2023 Hyperbee.AI Inc. All rights reserved. hello@hyperbee.ai

imported dataloader.py
NLP demo software by HyperbeeAI. Copyrights © 2023 Hyperbee.AI Inc. All rights reserved. hello@hyperbee.ai



In [2]:
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
BATCH_SIZE  = 48

In [3]:
SRC = Field(tokenize = tokenize_es, 
            init_token = tokenizer_es.token_to_id("<BOS>"), 
            eos_token = tokenizer_es.token_to_id("<EOS>"), 
            pad_token = tokenizer_es.token_to_id("<PAD>"),
            unk_token = tokenizer_es.token_to_id("<UNK>"),
            use_vocab = False,
            batch_first = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = tokenizer_en.token_to_id("<BOS>"), 
            eos_token = tokenizer_en.token_to_id("<EOS>"), 
            pad_token = tokenizer_en.token_to_id("<PAD>"),
            unk_token = tokenizer_en.token_to_id("<UNK>"),
            use_vocab = False,
            batch_first = True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print("Working with device:", device)

Working with device: cuda


In [4]:
train_data, valid_data, test_data = NewsDataset.splits(exts=('.es', '.en'), fields=(SRC, TRG))
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [5]:
enc   = encoder(device)
dec   = decoder(device, TRG_PAD_IDX)
model = seq2seq(enc, dec)

In [6]:
trained_checkpoint = "assets/es2en_hw_cp6.pt"
model.load_state_dict(torch.load(trained_checkpoint, map_location=device), strict=False);
model.to(device);

### serial conversion functions

In [7]:
def singlepass64_tensor2serial(seq_length, tensor):
    data       = tensor.cpu().detach().numpy();
    char_array = '';

    i=0;
    while i < 64:
        for j in range(0,seq_length):
            ch3 = data[0,i+3,j].astype('int8')
            ch2 = data[0,i+2,j].astype('int8')
            ch1 = data[0,i+1,j].astype('int8')
            ch0 = data[0,i+0,j].astype('int8')

            # 2s complements
            val3 = "{0:#0{1}x}".format(int(np.binary_repr(ch3, width=8), 2),4)
            val2 = "{0:#0{1}x}".format(int(np.binary_repr(ch2, width=8), 2),4)
            val1 = "{0:#0{1}x}".format(int(np.binary_repr(ch1, width=8), 2),4)
            val0 = "{0:#0{1}x}".format(int(np.binary_repr(ch0, width=8), 2),4)

            char_array += val3[2:] + val2[2:] + val1[2:] + val0[2:]

        i=i+4
        
    return char_array

def twos_comp(val, bits):
    if (val & (1 << (bits - 1))) != 0:
        val = val - (1 << bits)
    return val

def tensor_fromserial_singlepass64(char_array, seq_length, typetensor):
    out_tensor = torch.zeros_like(typetensor)
    i=0;
    while i < 64:
        for j in range(0, seq_length):
            cursor = (i*seq_length*2 + j*8); # seq_length*2 because we use 2 characters per element due to pyserial \CR \LF issue
            word   = char_array[cursor : cursor+8];
            
            # 2s complements
            val3 = twos_comp(int(word[0:2],16), 8)
            val2 = twos_comp(int(word[2:4],16), 8)
            val1 = twos_comp(int(word[4:6],16), 8)
            val0 = twos_comp(int(word[6:8],16), 8)
            
            out_tensor[0,i+3,j] = val3;
            out_tensor[0,i+2,j] = val2;
            out_tensor[0,i+1,j] = val1;
            out_tensor[0,i+0,j] = val0;
            
        i=i+4

    return out_tensor

def widemode_twos_comp(val, bits):
    if (val & (1 << (bits - 1))) != 0:
        val = ((val - (1 << bits)) >> 5) + 1
    return (val >> 5)

def tensor_fromserial_widemode64(char_array, seq_length, typetensor):
    out_tensor = torch.zeros_like(typetensor)
    i=0;
    while i < 64:
        for j in range(0, seq_length):
            cursor = (i*seq_length*8 + j*32); # seq_length*8 now because we use 8 characters per element, same pyserial issue
            word   = char_array[cursor : cursor+32];
            
            # 2s complements
            val0 = twos_comp(int(word[0:8],16),   32)
            val1 = twos_comp(int(word[8:16],16),  32)
            val2 = twos_comp(int(word[16:24],16), 32)
            val3 = twos_comp(int(word[24:32],16), 32)
            
            out_tensor[0,i+0,j] = val0;
            out_tensor[0,i+1,j] = val1;
            out_tensor[0,i+2,j] = val2;
            out_tensor[0,i+3,j] = val3;
            
        i=i+4

    return out_tensor

## Test

### choose id of example

In [8]:
example_idx = 120

### on PC

In [9]:
model.to(device)
src = vars(test_data.examples[example_idx])['src']
trg = tokenizer_en.decode(vars(test_data.examples[example_idx])['trg'], skip_special_tokens=False)
print(f'trg = {trg}')
print("")
translation = translate_sentence(src, SRC, TRG, model, device)
print(f'predicted trg = {translation}')
print("")
src = tokenizer_es.decode(src, skip_special_tokens=False)
print(f'src = {src}')
print("")

trg = but this won ’ t be the last answer , although for the time being it will drive corporate restructuring and the managerial mind .

predicted trg = but this will not be the latest response , though it will now be the central force of corporate restructuring and managerial thinking .

src = pero esto no será la última respuesta , aunque por ahora será la fuerza central de la reestructuración corporativa y el pensamiento gerencial .



### on chip

In [10]:
enc_pre = model.encoder.pre.to(device)
dec_pre = model.decoder.pre.to(device)
dec_i2w = model.decoder.fff.to(device)

src = vars(test_data.examples[example_idx])['src']
trg = tokenizer_en.decode(vars(test_data.examples[example_idx])['trg'], skip_special_tokens=False)

**MARK**

The below cell starts running a serial terminal on this notebook. First run this cell, and when it says "waiting for ai85", load the "assets/demo.elf" program onto the ai85 chip, and start running it (type c in gdb). This should trigger the terminal here, and operation should resume normally.

The cell is designed to translate a single sentence.

In [11]:
def ai85_demo_function():
    
    print("Please enter a Spanish sentence")
    textinput = input()
    print("")
    print("")

    src = (tokenizer_es.encode(textinput)).ids
    trg = tokenizer_en.decode(vars(test_data.examples[example_idx])['trg'], skip_special_tokens=False)
    with serial.Serial('/dev/ttyACM0', 115200) as ser: # , timeout=5 (not necessary, just for info)
        tokens     = src
        tokens     = [SRC.init_token] + tokens + [SRC.eos_token] + [SRC.pad_token] * (48 - 2 - len(tokens)) 
        src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)

        batch_size = src_tensor.shape[0];
        src_len    = src_tensor.shape[1];
        enc_pre_d  = enc_pre(src_tensor, 0, src_len, batch_size);
        encarray   = singlepass64_tensor2serial(48, enc_pre_d);

        #### to chip
        print("** shallow.AI ai85 demo **")
        print("** loading demo to ai85 **")
        line = ser.readline()
        while(line != b''):
            line = ser.readline()
            if(line == b'GJcav7Wf2kmhaXJdsO0QVzX3slsv96Ck\r\n'):
                ser.write(encarray.encode(encoding="ascii"))
                line = ser.readline()
                break

        trg_indexes = [TRG.init_token, ] + [TRG.pad_token] * (48 - 1) 

        done_decoding_flag = False
        for i in range(47):
            start_idx = max(0, i - 7)
            trg_tensor = torch.LongTensor(trg_indexes[start_idx:start_idx + 8]).unsqueeze(0).to(device)
            batch_size   = trg_tensor.shape[0]
            trg_len      = trg_tensor.shape[1]
            pos_start    = max(0, i - 7)
            dec_pre_d    = dec_pre(trg_tensor, pos_start, trg_len + pos_start, batch_size)
            decarray     = singlepass64_tensor2serial(8, dec_pre_d);
            while(line != b''):
                line = ser.readline()
                if(line == b'gZMFxLf6muLVf9P6Iyea56VbA4qktpUR\r\n'):
                    if(done_decoding_flag):
                        print("****** ai85 is done ******")
                        decarray = "done" + decarray[4:]
                    ser.write(decarray.encode(encoding="ascii"))
                    line = ser.readline()
                    break

            if(done_decoding_flag):
                break

            line = ser.readline()
            h2e_out = tensor_fromserial_widemode64(line, 1, dec_pre_d[:,:,0:1]) / (128.0 * 2**(5+1))
            output  = dec_i2w(h2e_out.permute(0, 2, 1))
            pred_token = output.argmax(2)
            trg_indexes[i + 1] = pred_token
            if pred_token == TRG.eos_token:
                done_decoding_flag = True
                
    try:
        trg_indexes = trg_indexes[1:trg_indexes.index(TRG.eos_token)]
    except ValueError: 
        trg_indexes = trg_indexes[1:]

    trg_tokens = tokenizer_en.decode(trg_indexes, skip_special_tokens=False)
    
    print("")
    print("")
    print("English translation on ai85:")
    print(f'{trg_tokens}')

# NLP demo software by HyperbeeAI

Copyrights © 2023 Hyperbee.AI Inc. All rights reserved. hello@hyperbee.ai 

In [12]:
ai85_demo_function()

Please enter a Spanish sentence
La vinculación entre el crecimiento económico y el bienestar humano parece evidente.


** shallow.AI ai85 demo **
** loading demo to ai85 **
****** ai85 is done ******


English translation on ai85:
the link between economic growth and human welfare seems clear .
